# Dual BLE: Heart Rate + Sterzo Example

This is an advanced demonstration that builds on [`single_hr_notebook_example.ipynb`](single_hr_notebook_example.ipynb) to demonstrate streaming from two devices simultaneously.

This examples uses Bluetooth Low Energy (BLE) Heart Rate service and BLE Sterzo.
+ It must support Bluetooth, not just ANT+
+ The GATT Characteristic required is: `00002a37-0000-1000-8000-00805f9b34fb`
+ You may use the [`bluetooth_dissect.py` tool](https://github.com/tensorturtle/bluetooth_dissect) to check if your device supports that GATT Characteristic.

Example compatible Heart Rate devices:
+ Magene H603
+ Wahoo TICKR & TICKR Fit
+ WHOOP 4.0 (Broadcast Heart Rate ON)
+ and many more advertised as "supports Bluetooth Heart Rate"

Compatible Sterzo device:
+ [Elite Sterzo Smart Steering Plate](https://www.elite-it.com/en/products/home-trainers/ecosystem-accessories/sterzo-smart)

In [ ]:
!pip3 install -r -q requirements.txt

# Setup

In [ ]:
import asyncio

In [ ]:
from bleak_fsm import machine, BleakModel

In [ ]:
hr_model = BleakModel()
sterzo_model = BleakModel()

machine.add_model(hr_model)
machine.add_model(sterzo_model)

Toggle this `USE_PYCYCLING` flag to see how this library can accomodate using standard, raw BLeakClient or Pycycling-wrapped BleakClient.

In [ ]:
# Define callbacks

from pycycling.heart_rate_service import HeartRateService
hr_model.wrap = lambda client: HeartRateService(client)
hr_model.enable_notifications = lambda client: client.enable_hr_measurement_notifications()
hr_model.disable_notifications = lambda client: client.disable_hr_measurement_notifications()
def handle_hr_measurement(value):
    print("Using Pycycling wrapper around BleakClient")
    print(f"Heart Rate: {value}")
hr_model.set_measurement_handler = lambda client: client.set_hr_measurement_handler(handle_hr_measurement)

from pycycling.sterzo import Sterzo
sterzo_model.wrap = lambda client: Sterzo(client)
sterzo_model.enable_notifications = lambda client: client.enable_steering_measurement_notifications()
sterzo_model.disable_notifications = lambda client: client.disable_steering_measurement_notifications()
def handle_steer_measurement(value):
    print("Using Pycycling wrapper around BleakClient")
    print(f"Sterzo: {value}")
sterzo_model.set_measurement_handler = lambda client: client.set_steering_measurement_callback(handle_steer_measurement)


# Usage

We can use any of the two models to call `start_scan()`, and the results will be shared among them through their shared class variable.

In [ ]:
await BleakModel.start_scan()

In [ ]:
await asyncio.sleep(3)

In [ ]:
await BleakModel.stop_scan()

In [ ]:
BleakModel.bt_devices

Copy-and-paste the key corresponding to the device that you want to connect to:

In [ ]:
hr_target_address = "14863C4D-BF71-4EA3-C6B4-98001056AAF8"
sterzo_target_address = "715D8603-DC4C-2994-C0CD-2BC5A93E0B38"

In [ ]:
hr_model.state

In [ ]:
sterzo_model.state

In [ ]:
await hr_model.set_target(hr_target_address)
await sterzo_model.set_target(sterzo_target_address)

In [ ]:
hr_model.state

In [ ]:
sterzo_model.state

In [ ]:
await hr_model.connect()

In [ ]:
await sterzo_model.connect()

Sidebar: Note how the above cell returns `True`. All transition methods on `model` returns True if the transition was successful. Therefore you may explicitly define happy-path and exception-path in different ways.

For debugging,
```python
assert await model.connect(), "Model failed to connect"
```

Or, more properly check the return value
```python
success = await model.connect()
if not succcess:
    await model.clean_up() # this catch-all method brings the model back to "Init"
    raise Exception("Failed to connect")
```

Or, we can check the `.state` afterwards, like so:

In [ ]:
if hr_model.state != "Connected" or sterzo_model.state != "Connected":
    raise Exception("Failed to connect. If the device was not properly disconnected last time, restart either the device or bluetooth adapter")

In order to receive/send actual data from/to the connected device, we need to start a stream.

This will run forever until `model.disconnect()` is called, so for now let's sleep for several seconds and then call that.

In [ ]:
await hr_model.stream()

In [ ]:
await sterzo_model.stream()

Look through the output of the next cell. You should see some heart rate data sprinkled among sterzo data.

In [ ]:
await asyncio.sleep(10)

Bleak-FSM encapsulates a kind of bug in Bleak, where it is not possible to re-use a connected client. 

That is, we shouldn't step back to "Connected" state from "Streaming", because we can't go from "Connected" back to "Streaming".

Instead we need to go back two steps to "TargetSet", re-establish a connection, and then we can stream again.

Bleak-FSM enforces this behavior for your own safety.
There exists not method for you to even try to go from "Streaming" to "Connected".


In [ ]:
await hr_model.clean_up()

In [ ]:
await sterzo_model.clean_up()

Whenever the application terminates, or otherwise no longer needs to communicate with the bluetooth device, it should always call `disconnect()` on the model. The model state should always e either "TargetSet" or "Init".